In [1]:
from sklearn.cluster import KMeans,MeanShift, estimate_bandwidth , Birch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import scipy.sparse as sp
import igraph as ig
import leidenalg as la
import pandas as pd
from collections import Counter
from sklearn.metrics import adjusted_rand_score, rand_score,normalized_mutual_info_score

In [2]:
df = pd.read_csv("some-reddit.csv")
df

,rotoreuters,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,fiplefip,11.809120,-0.236315,9.712246,-3.558550,0.676302,-0.330307,-2.824460,-0.289928,0.484788,-3.428256
1,amici_ursi,11.039474,0.714586,7.176633,-6.500434,1.621503,0.676836,-3.538689,-0.039432,2.002969,0.370842
2,unremovable,12.308345,2.944823,13.467455,-6.039847,-1.338466,-0.693709,-2.730207,-3.215488,-1.463924,0.722347
3,CDRE_64,9.597274,-0.275656,9.219146,-5.203834,-0.475436,0.101921,-3.843878,-0.283923,1.681646,0.044997
4,noeatnosleep,8.441664,-1.520713,2.302667,-4.226753,1.043632,1.490670,-6.036377,0.880610,-0.303432,0.104033
5,PostNationalism,5.013857,-2.501058,3.895118,2.235447,3.307937,-0.196699,1.413836,-2.910550,-0.214350,-0.770090
6,trueworldnews,12.048599,1.202438,6.710602,-3.770383,1.228732,-1.718225,-2.476334,-2.163013,3.820246,1.960525
7,drewiepoodle,8.218326,-2.528193,4.460269,-0.936959,-1.685539,-4.491433,-3.246064,-2.502605,-1.823646,1.836632
8,RIPmod,20.111155,2.807410,19.705938,-5.255570,-0.362161,-4.346634,-0.508715,-7.902928,2.057837,1.270654
9,michaelconfoy,4.567328,-8.470312,8.456068,0.283074,-2.701900,-0.641383,1.128566,1.066758,-4.142665,0.631451


In [3]:
X = df.iloc[:, 1:].values

In [4]:
k = 2

nbrs = NearestNeighbors(n_neighbors=k, metric='cosine').fit(X)
distances, indices = nbrs.kneighbors(X)

In [5]:
rows = np.repeat(np.arange(len(X)), k)
cols = indices.flatten()
vals = 1 - distances.flatten()        # تبدیل distance → similarity

A = sp.csr_matrix((vals, (rows, cols)), shape=(len(X), len(X)))

In [6]:
A

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 98 stored elements and shape (49, 49)>

In [7]:
A = A.maximum(A.T)
# گراف undirected

In [8]:
# تبدیل sparse matrix به گراف igraph
sources, targets = A.nonzero()
weights = A.data

g = ig.Graph(n=A.shape[0], edges=list(zip(sources, targets)))
g.es["weight"] = weights

# اجرای leiden
partition = la.find_partition(g, la.RBConfigurationVertexPartition, weights="weight")
partition

In [9]:
print(partition)

Clustering with 49 elements and 10 clusters
[ 0] 7, 15, 16, 17, 20, 37, 39, 40, 47
[ 1] 0, 1, 3, 4, 22, 31, 48
[ 2] 6, 12, 25, 34, 35, 38, 45
[ 3] 10, 21, 26, 32, 46
[ 4] 13, 24, 33, 42, 43
[ 5] 5, 11, 23, 30
[ 6] 2, 14, 28, 44
[ 7] 8, 18, 19
[ 8] 9, 29, 41
[ 9] 27, 36


In [11]:
labels_pred = partition.membership
print(labels_pred)

[1, 1, 6, 1, 1, 5, 2, 0, 7, 8, 3, 5, 2, 4, 6, 0, 0, 0, 7, 7, 0, 3, 1, 5, 4, 2, 3, 9, 6, 8, 5, 1, 3, 4, 2, 2, 9, 0, 2, 0, 0, 8, 4, 4, 6, 2, 3, 0, 1]


In [12]:
df["leiden_cluster"] = labels_pred
df

,rotoreuters,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,leiden_cluster
0,fiplefip,11.809120,-0.236315,9.712246,-3.558550,0.676302,-0.330307,-2.824460,-0.289928,0.484788,-3.428256,1
1,amici_ursi,11.039474,0.714586,7.176633,-6.500434,1.621503,0.676836,-3.538689,-0.039432,2.002969,0.370842,1
2,unremovable,12.308345,2.944823,13.467455,-6.039847,-1.338466,-0.693709,-2.730207,-3.215488,-1.463924,0.722347,6
3,CDRE_64,9.597274,-0.275656,9.219146,-5.203834,-0.475436,0.101921,-3.843878,-0.283923,1.681646,0.044997,1
4,noeatnosleep,8.441664,-1.520713,2.302667,-4.226753,1.043632,1.490670,-6.036377,0.880610,-0.303432,0.104033,1
5,PostNationalism,5.013857,-2.501058,3.895118,2.235447,3.307937,-0.196699,1.413836,-2.910550,-0.214350,-0.770090,5
6,trueworldnews,12.048599,1.202438,6.710602,-3.770383,1.228732,-1.718225,-2.476334,-2.163013,3.820246,1.960525,2
7,drewiepoodle,8.218326,-2.528193,4.460269,-0.936959,-1.685539,-4.491433,-3.246064,-2.502605,-1.823646,1.836632,0
8,RIPmod,20.111155,2.807410,19.705938,-5.255570,-0.362161,-4.346634,-0.508715,-7.902928,2.057837,1.270654,7
9,michaelconfoy,4.567328,-8.470312,8.456068,0.283074,-2.701900,-0.641383,1.128566,1.066758,-4.142665,0.631451,8


In [13]:
labels_true = df["leiden_cluster"].values
labels_true

array([1, 1, 6, 1, 1, 5, 2, 0, 7, 8, 3, 5, 2, 4, 6, 0, 0, 0, 7, 7, 0, 3,
       1, 5, 4, 2, 3, 9, 6, 8, 5, 1, 3, 4, 2, 2, 9, 0, 2, 0, 0, 8, 4, 4,
       6, 2, 3, 0, 1])

#### Rand Index , Adjusted Rand Index (ARI) , Modularity ,NMI (Normalized Mutual Information) , Purity

In [14]:

class EvaluationMetrics:
    def __init__(self, graph=None, partition=None):
        self.graph = graph
        self.partition = partition

        self.y_true = None
        self.y_pred = None

        self._purity = None
        self._nmi = None
        self._ari = None
        self._ri = None
        self._modularity = None

    # Purity
    def _compute_purity(self):
        y_true = self.y_true
        y_pred = self.y_pred

        clusters = np.unique(y_pred)
        N = len(y_true)
        total = 0

        for c in clusters:
            idx = np.where(y_pred == c)[0]
            true_labels = y_true[idx]
            if len(true_labels) == 0:
                continue
            most_common = Counter(true_labels).most_common(1)[0][1]
            total += most_common

        return total / N

    # MAIN FIT
    def fit(self, y_true, y_pred):
        self.y_true = np.asarray(y_true)
        self.y_pred = np.asarray(y_pred)

        # purity
        self._purity = self._compute_purity()

        # nmi
        self._nmi = normalized_mutual_info_score(self.y_true, self.y_pred)

        # ari
        self._ari = adjusted_rand_score(self.y_true, self.y_pred)

        # ri
        self._ri = rand_score(self.y_true, self.y_pred)

        # modularity (نیازمند گراف + پارتیشن)
        if self.graph is not None and self.partition is not None:
            try:
                self._modularity = self.graph.modularity(self.partition.membership, weights="weight")
            except:
                self._modularity = None

        return self

    # GETTERS
    def score_purity(self):
        return self._purity

    def score_nmi(self):
        return self._nmi

    def score_ari(self):
        return self._ari

    def score_ri(self):
        return self._ri

    def score_modularity(self):
        return self._modularity

In [15]:
metrics = EvaluationMetrics(graph=g, partition=partition)

metrics.fit(labels_true, labels_pred)

print("Purity:", metrics.score_purity())
print("NMI:", metrics.score_nmi())
print("ARI:", metrics.score_ari())
print("RI:", metrics.score_ri())
print("Modularity:", round(metrics.score_modularity(),3))

Purity: 1.0
NMI: 1.0
ARI: 1.0
RI: 1.0
Modularity: 0.819


<div lang="fa" dir="rtl" style="
  font-family: 'Vazirmatn','IRANSans','Tahoma',sans-serif;
  line-height: 1.9;
  color: #222;
  background: #787;
  border: 1px solid #e5e5e5;
  border-radius: 12px;
  padding: 18px 20px;
  box-shadow: 0 6px 20px rgba(0,0,0,0.06);
  margin: 12px 0;
">
    <li>ایجاد نویز در لیبل ها</li>

In [23]:
labels_pred_noisy = np.array(labels_pred).copy()
labels_pred_noisy[:20] = labels_pred_noisy[:20] + 1
labels_pred_noisy = labels_pred_noisy.tolist()

In [24]:
metrics.fit(labels_true, labels_pred_noisy)

print("Purity:", round(metrics.score_purity(),3))
print("NMI:", round(metrics.score_nmi(),3))
print("ARI:", round(metrics.score_ari(),3))
print("RI:", round(metrics.score_ri(),4))
print("Modularity:", round(metrics.score_modularity(),3))

Purity: 0.673
NMI: 0.748
ARI: 0.436
RI: 0.8997
Modularity: 0.819
